In [1]:
#Import and install tools

!pip install pandasql
import os
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as sm
import numpy as np
import matplotlib.pyplot as plt
from sqlite3 import connect
from scipy.stats.mstats import winsorize, ks_2samp
import seaborn as sns
import pandasql as ps

conn = connect(':memory:', timeout = 10)
filePath = "/Users/aaron/AaronTuFIMA/FamaFrenchFiveFactors"

In [2]:
# import CRSP data
file = filePath + "/CRSPMonthly1990Through2022.csv"
crsp1990To2022 = pd.read_csv(file)
print(crsp1990To2022)
print(crsp1990To2022.columns)

C:\Users\aaron\AppData\Local\Temp\ipykernel_2200\2350463996.py:3: DtypeWarning: Columns (5,9,18,19,49,58) have mixed types. Specify dtype option on import or set low_memory=False.
  crsp1990To2022 = pd.read_csv(file)


         PERMNO      date  NAMEENDT  SHRCD  EXCHCD   SICCD    NCUSIP TICKER  \
0         10001  19900131       NaN   11.0     3.0    4920  39040610   GFGC   
1         10001  19900228       NaN   11.0     3.0    4920  39040610   GFGC   
2         10001  19900330       NaN   11.0     3.0    4920  39040610   GFGC   
3         10001  19900430       NaN   11.0     3.0    4920  39040610   GFGC   
4         10001  19900531       NaN   11.0     3.0    4920  39040610   GFGC   
...         ...       ...       ...    ...     ...     ...       ...    ...   
2975236   93436  20211130       NaN   11.0     3.0  9999.0  88160R10   TSLA   
2975237   93436  20211231       NaN   11.0     3.0  9999.0  88160R10   TSLA   
2975238   93436  20220131       NaN   11.0     3.0  9999.0  88160R10   TSLA   
2975239   93436  20220228       NaN   11.0     3.0  9999.0  88160R10   TSLA   
2975240   93436  20220331       NaN   11.0     3.0  9999.0  88160R10   TSLA   

                     COMNAM SHRCLS  ... CFACSHR    

In [3]:
#Clean CRSP Dataframe

#Parse relevant variables
crsp1990To2022.to_sql("crsp1990To2022", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT PERMNO as permno, date, RET as ret, SHROUT as shrout, ALTPRC as altprc, EXCHCD as exchcd,
        SHRCD as shrcd, SICCD as siccd, DLRET as dlret, DLSTCD as dlstcd
        FROM crsp1990To2022
        '''
crsp1990To2022 = pd.read_sql(query, conn)

In [4]:
#Convert each column to correct variable types
crsp1990To2022['permno'] = pd.to_numeric(crsp1990To2022['permno'], errors = 'coerce') #security identifier
crsp1990To2022['date'] = pd.to_datetime(crsp1990To2022['date'], format = '%Y%m%d') #month identifier
crsp1990To2022['ret'] = pd.to_numeric(crsp1990To2022['ret'], errors = 'coerce') #return
crsp1990To2022['shrout'] = pd.to_numeric(crsp1990To2022['shrout'], errors = 'coerce') #share outstanding (thousands)
crsp1990To2022['altprc'] = pd.to_numeric(crsp1990To2022['altprc'], errors = 'coerce') #last traded price in a month
crsp1990To2022['exchcd'] = pd.to_numeric(crsp1990To2022['exchcd'], errors = 'coerce') #exchange code
crsp1990To2022['shrcd'] = pd.to_numeric(crsp1990To2022['shrcd'], errors = 'coerce') #share code
crsp1990To2022['siccd'] = pd.to_numeric(crsp1990To2022['siccd'], errors = 'coerce') #industry code
crsp1990To2022['dlret'] = pd.to_numeric(crsp1990To2022['dlret'], errors = 'coerce') #delisting return
crsp1990To2022['dlstcd'] = pd.to_numeric(crsp1990To2022['dlstcd'], errors = 'coerce') #delisting code

In [5]:
#Convert returns to percent
crsp1990To2022['ret'] *= 100
crsp1990To2022['dlret'] *= 100

#only keep data after 2018
crsp2018To2022 = crsp1990To2022[crsp1990To2022['date'] >= "2018-01-01"]


In [6]:
#Only keep US-based common stocks
crsp2018To2022 = crsp2018To2022[(crsp2018To2022['shrcd'] == 10) | (crsp2018To2022['shrcd'] == 11)]

#calculate market cap (millions)
crsp2018To2022['market_cap'] = abs(crsp2018To2022['shrout'] * crsp2018To2022['altprc']) / 1000
crsp2018To2022['market_cap'].replace(0, np.NaN)

2358       2.585782e+03
2359       2.509016e+03
2360       2.553262e+03
2361       2.569842e+03
2362       2.648577e+03
               ...     
2947235    1.149642e+06
2947236    1.092218e+06
2947237    9.681319e+05
2947238    8.996190e+05
2947239    1.113736e+06
Name: market_cap, Length: 194624, dtype: float64

In [7]:
# Add exchange column, define exchange labels and keep only relevant labels
crsp2018To2022['exchange'] = ""
for i in crsp2018To2022.index:
    if crsp2018To2022['exchcd'][i] in [1, 31]:
        crsp2018To2022.loc[i, 'exchange'] = "NYSE"
    elif crsp2018To2022['exchcd'][i] in [2, 32]:
        crsp2018To2022.loc[i, 'exchange'] = "AMEX"
    elif crsp2018To2022['exchcd'][i] in [3, 33]:
        crsp2018To2022.loc[i, 'exchange'] = "NASDAQ"
    else:
        crsp2018To2022.loc[i, 'exchange'] = "Other"

crsp2018To2022 = crsp2018To2022[crsp2018To2022['exchange'] != "Other"]

In [8]:
#add adjusted Return Column, adjust delisting returns
crsp2018To2022['ret_adj'] = ""
for i in crsp2018To2022.index:
    if pd.isnull(crsp2018To2022['dlstcd'][i]):
        crsp2018To2022.loc[i, 'ret_adj'] = crsp2018To2022.loc[i, 'ret']
    elif (pd.notnull(crsp2018To2022['dlstcd'][i])) and (pd.notnull(crsp2018To2022['dlret'][i])):
        crsp2018To2022.loc[i, 'ret_adj'] = crsp2018To2022.loc[i, 'dlret']
    elif (551 <= crsp2018To2022.loc[i, 'dlstcd'] <= 574) or (crsp2018To2022.loc[i, 'dlstcd'] in [500, 520, 580, 584]):
        crsp2018To2022.loc[i, 'ret_adj'] = -30
    else:
        crsp2018To2022.loc[i, 'ret_adj'] = -100

In [9]:
#Reindex and drop unneeded columns
crsp2018To2022 = crsp2018To2022.reset_index()
crsp2018To2022.drop(['shrcd', 'dlret', 'dlstcd', 'index'], inplace = True, axis = 1)

print(crsp2018To2022)
print(crsp2018To2022.columns)

        permno       date      ret     shrout      altprc  exchcd   siccd  \
0        10026 2018-01-31  -8.8191    18678.0   138.44000     3.0  2052.0   
1        10026 2018-02-28  -2.9688    18678.0   134.33000     3.0  2052.0   
2        10026 2018-03-29   1.9951    18697.0   136.56000     3.0  2052.0   
3        10026 2018-04-30   0.6224    18702.0   137.41000     3.0  2052.0   
4        10026 2018-05-31   3.0638    18702.0   141.62000     3.0  2052.0   
...        ...        ...      ...        ...         ...     ...     ...   
193370   93436 2021-11-30   2.7612  1004265.0  1144.76001     3.0  9999.0   
193371   93436 2021-12-31  -7.6855  1033534.0  1056.78003     3.0  9999.0   
193372   93436 2022-01-31 -11.3609  1033534.0   936.71997     3.0  9999.0   
193373   93436 2022-02-28  -7.0768  1033534.0   870.42999     3.0  9999.0   
193374   93436 2022-03-31  23.8009  1033534.0  1077.59998     3.0  9999.0   

          market_cap exchange  ret_adj  
0       2.585782e+03   NASDAQ  -8.

In [10]:
#Import CRSP/Compustat Merged (CCM) Data

file = filePath + "/ccmfund1990to2022.csv"
ccmFund1990To2022 = pd.read_csv(file)
print(ccmFund1990To2022)
print(ccmFund1990To2022.columns)

C:\Users\aaron\AppData\Local\Temp\ipykernel_2200\2341482942.py:4: DtypeWarning: Columns (2,17,19,24,33,37,952,953,955,956,957,962,976,977,982) have mixed types. Specify dtype option on import or set low_memory=False.
  ccmFund1990To2022 = pd.read_csv(file)


         GVKEY LINKPRIM LIID LINKTYPE  LPERMNO  LPERMCO    LINKDT LINKENDDT  \
0         1004        P    1       LU    54594    20000  19720424         E   
1         1004        P    1       LU    54594    20000  19720424         E   
2         1004        P    1       LU    54594    20000  19720424         E   
3         1004        P    1       LU    54594    20000  19720424         E   
4         1004        P    1       LU    54594    20000  19720424         E   
...        ...      ...  ...      ...      ...      ...       ...       ...   
209121  345980        P    1       LC    20333    57666  20201216         E   
209122  345980        P    1       LC    20333    57666  20201216         E   
209123  347007        P    1       LC    15533    55364  20210310         E   
209124  347085        P    1       LC    21069    58208  20210401         E   
209125  349530        P    1       LC    17324    56274  20210709         E   

        datadate   fyear  ... priusa     sic spcind

In [11]:
#Clean CRSP/Compustat merged Dataframe

#parse relevant Variables
ccmFund1990To2022.to_sql("ccmFund1990To2022", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT GVKEY, LPERMNO, datadate, LINKTYPE, LINKENDDT, seq, ceq, at, lt, txditc,
                        txdb, itcb, pstkrv, pstkl, pstk, indfmt, datafmt
        FROM ccmFund1990To2022
        '''
ccmFund1990To2022 = pd.read_sql(query, conn)

In [12]:
#Convert Variables
ccmFund1990To2022['GVKEY'] = pd.to_numeric(ccmFund1990To2022['GVKEY'], errors = 'coerce') #firm identifier
ccmFund1990To2022['LPERMNO'] = pd.to_numeric(ccmFund1990To2022['LPERMNO'], errors = 'coerce') #stock identifier
ccmFund1990To2022['datadate'] = pd.to_datetime(ccmFund1990To2022['datadate'], format = '%Y%m%d') #date of report
ccmFund1990To2022['LINKTYPE'] = ccmFund1990To2022['LINKTYPE'].apply(str) #link type
ccmFund1990To2022['LINKENDDT'] = pd.to_datetime(ccmFund1990To2022['LINKENDDT'], format = '%Y%m%d', errors = 'coerce') #link's date of expiration
ccmFund1990To2022['seq'] = pd.to_numeric(ccmFund1990To2022['seq'], errors = 'coerce') #stockholder's equity
ccmFund1990To2022['ceq'] = pd.to_numeric(ccmFund1990To2022['ceq'], errors = 'coerce') #total equity
ccmFund1990To2022['at'] = pd.to_numeric(ccmFund1990To2022['at'], errors = 'coerce') #total assets
ccmFund1990To2022['lt'] = pd.to_numeric(ccmFund1990To2022['lt'], errors = 'coerce') #total liabilities
ccmFund1990To2022['txditc'] = pd.to_numeric(ccmFund1990To2022['txditc'], errors = 'coerce') #deferred taxes and investment tax credit
ccmFund1990To2022['txdb'] = pd.to_numeric(ccmFund1990To2022['txdb'], errors = 'coerce') #deferred taxes
ccmFund1990To2022['itcb'] = pd.to_numeric(ccmFund1990To2022['itcb'], errors = 'coerce') #investment tax credit
ccmFund1990To2022['pstkrv'] = pd.to_numeric(ccmFund1990To2022['pstkrv'], errors = 'coerce') #preferred stock redemption value
ccmFund1990To2022['pstkl'] = pd.to_numeric(ccmFund1990To2022['pstkl'], errors = 'coerce') #preferred stock liquidating value
ccmFund1990To2022['pstk'] = pd.to_numeric(ccmFund1990To2022['pstk'], errors = 'coerce') #preffered stock par value
ccmFund1990To2022['indfmt'] = ccmFund1990To2022['indfmt'].apply(str) #industry format
ccmFund1990To2022['datafmt'] = ccmFund1990To2022['datafmt'].apply(str) #data format

#Filter Columns

#keep only data after 2018
ccmFund2018To2022 = ccmFund1990To2022[ccmFund1990To2022['datadate'] >= "2018-01-01 00:00:00"]

#Only keep correct formats
ccmFund2018To2022 = ccmFund2018To2022[ccmFund2018To2022['indfmt'] == "INDL"]
ccmFund2018To2022 = ccmFund2018To2022[ccmFund2018To2022['datafmt'] == "STD"]

#Only keep valid links
ccmFund2018To2022 = ccmFund2018To2022[(ccmFund2018To2022['LINKTYPE'] == "LU") | (ccmFund2018To2022['LINKTYPE'] == "LC")]

#Only keep links active at datadate
ccmFund2018To2022 = ccmFund2018To2022[(ccmFund2018To2022['datadate'] <= ccmFund2018To2022['LINKENDDT']) | pd.isnull(ccmFund2018To2022['LINKENDDT'])]

In [13]:
#Calculate book value
ccmFund2018To2022.to_sql("ccmFund2018To2022", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT GVKEY as gvkey, LPERMNO as permno, datadate,
                        COALESCE(seq, ceq + pstk, at - lt) + COALESCE(txditc, txdb + itcb, 0) - COALESCE(pstkrv, pstkl, pstk, 0) as book_value
        FROM ccmFund2018To2022
        '''
ccmFund2018To2022 = pd.read_sql(query, conn)
ccmFund2018To2022['book_value'] = pd.to_numeric(ccmFund2018To2022['book_value'], errors = 'coerce')

In [14]:
#Only keep the last observation of each year
ccmFund2018To2022['datadate'] = pd.to_datetime(ccmFund2018To2022['datadate'])
ccmFund2018To2022['year'] = ccmFund2018To2022['datadate'].dt.year
ccmFund2018To2022 = ccmFund2018To2022.sort_values(by = ['permno', 'datadate'], ascending = True)
ccmFund2018To2022 = ccmFund2018To2022.reset_index()
toDrop = []
for i in range(len(ccmFund2018To2022) - 1):
    if(ccmFund2018To2022.loc[i, 'permno'] == ccmFund2018To2022.loc[i + 1, 'permno']) and (ccmFund2018To2022.loc[i, 'year'] == ccmFund2018To2022.loc[i + 1, 'year']):
        toDrop.append(i)
ccmFund2018To2022.drop(toDrop, inplace = True, axis = 0)
ccmFund2018To2022.drop('index', inplace = True, axis = 1)

In [15]:
#Drop missing/infinite values
ccmFund2018To2022.replace([np.inf, -np.inf], np.nan, inplace = True)
ccmFund2018To2022 = ccmFund2018To2022.dropna()

#Reindex
ccmFund2018To2022 = ccmFund2018To2022.reset_index()
ccmFund2018To2022.drop('index', inplace = True, axis = 1)

In [16]:
#Add reference date for matching (6/1 of next calendar year)
ccmFund2018To2022['year'] = pd.to_numeric(ccmFund2018To2022['year'], errors = 'coerce')
ccmFund2018To2022['reference_date'] = ccmFund2018To2022['year'] + 1
ccmFund2018To2022['reference_date'] = ccmFund2018To2022['reference_date'].apply(str)
ccmFund2018To2022['reference_date'] += "-06-01"
ccmFund2018To2022['reference_date'] = pd.to_datetime(ccmFund2018To2022['reference_date'])
ccmFund2018To2022.drop('year', inplace = True, axis = 1)

print(ccmFund2018To2022)
print(ccmFund2018To2022.columns)

        gvkey  permno   datadate  book_value reference_date
0       12825   10026 2018-09-30     811.413     2019-06-01
1       12825   10026 2019-09-30     895.671     2020-06-01
2       12825   10026 2020-09-30     873.911     2021-06-01
3       12825   10026 2021-09-30     907.232     2022-06-01
4       12096   10028 2018-12-31       8.413     2019-06-01
...       ...     ...        ...         ...            ...
20524  184259   93434 2021-06-30      74.602     2022-06-01
20525  184996   93436 2018-12-31    4923.243     2019-06-01
20526  184996   93436 2019-12-31    6618.000     2020-06-01
20527  184996   93436 2020-12-31   22376.000     2021-06-01
20528  184996   93436 2021-12-31   30213.000     2022-06-01

[20529 rows x 5 columns]
Index(['gvkey', 'permno', 'datadate', 'book_value', 'reference_date'], dtype='object')


In [17]:
# Construct stocks sample

#Parse relevant variables from CRSP
crsp2018To2022.to_sql("crsp2018To2022", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT permno, date, exchange, ret_adj as ret, market_cap
        FROM crsp2018To2022
        '''
stocks2018To2022 = pd.read_sql(query, conn)

#Drop missing/infinite values
stocks2018To2022.replace([np.inf, -np.inf], np.nan, inplace = True)
stocks2018To2022 = stocks2018To2022.dropna()

In [18]:
#Add reference date for each stock (last 6/1)
stocks2018To2022['date'] = pd.to_datetime(stocks2018To2022['date'])
stocks2018To2022['reference_date'] = ""
for i in stocks2018To2022.index:
    if stocks2018To2022.loc[i, 'date'].month < 6:
        stocks2018To2022.loc[i, 'reference_date'] = str(stocks2018To2022.loc[i, 'date'].year - 1) + "-06-01"
    else:
        stocks2018To2022.loc[i, 'reference_date'] = str(stocks2018To2022.loc[i, 'date'].year) + "-06-01"
stocks2018To2022['reference_date'] = pd.to_datetime(stocks2018To2022['reference_date'])

In [19]:
#Merge compustat data into stocks data
stocks2018To2022.to_sql("stocks2018To2022", conn, if_exists = 'replace')
ccmFund2018To2022.to_sql("ccmFund2018To2022", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT stocks2018To2022.*, ccmFund2018To2022.gvkey, ccmFund2018To2022.datadate,
                        ccmFund2018To2022.book_value
        FROM stocks2018To2022
        LEFT JOIN ccmFund2018To2022
        ON stocks2018To2022.permno = ccmFund2018To2022.permno
        AND stocks2018To2022.reference_date = ccmFund2018To2022.reference_date
        '''
stocks2018To2022 = pd.read_sql(query, conn)
stocks2018To2022.drop('index', inplace = True, axis = 1)

In [20]:
print(stocks2018To2022)

        permno                 date exchange       ret    market_cap  \
0        10026  2018-01-31 00:00:00   NASDAQ   -8.8191  2.585782e+03   
1        10026  2018-02-28 00:00:00   NASDAQ   -2.9688  2.509016e+03   
2        10026  2018-03-29 00:00:00   NASDAQ    1.9951  2.553262e+03   
3        10026  2018-04-30 00:00:00   NASDAQ    0.6224  2.569842e+03   
4        10026  2018-05-31 00:00:00   NASDAQ    3.0638  2.648577e+03   
...        ...                  ...      ...       ...           ...   
191612   93436  2021-11-30 00:00:00   NASDAQ    2.7612  1.149642e+06   
191613   93436  2021-12-31 00:00:00   NASDAQ   -7.6855  1.092218e+06   
191614   93436  2022-01-31 00:00:00   NASDAQ  -11.3609  9.681319e+05   
191615   93436  2022-02-28 00:00:00   NASDAQ   -7.0768  8.996190e+05   
191616   93436  2022-03-31 00:00:00   NASDAQ      -100  1.113736e+06   

             reference_date     gvkey             datadate  book_value  
0       2017-06-01 00:00:00       NaN                 None    

In [21]:
stocks2018To2022['date'] = pd.to_datetime(stocks2018To2022['date'])

# Filter the DataFrame for rows with month equal to 12
stocksMarketEquity = stocks2018To2022[stocks2018To2022['date'].dt.month == 12].copy()

# Create the 'reference_date' column by adding 1 year to the 'date' column
stocksMarketEquity['reference_date'] = stocksMarketEquity['date'].dt.year + 1
stocksMarketEquity['reference_date'] = stocksMarketEquity['reference_date'].apply(str)
stocksMarketEquity['reference_date'] += "-06-01"
stocksMarketEquity['reference_date'] = pd.to_datetime(stocksMarketEquity['reference_date'])


In [22]:
stocksMarketEquity.to_sql("stocksMarketEquity", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT permno, reference_date, market_cap as market_equity
        FROM stocksMarketEquity
        '''
stocksMarketEquity = pd.read_sql(query, conn)
stocks2018To2022.to_sql("stocks2018To2022", conn, if_exists = 'replace')
stocksMarketEquity.to_sql("stocksMarketEquity", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT stocks2018To2022.*, stocksMarketEquity.market_equity
        FROM stocks2018To2022
        LEFT JOIN stocksMarketEquity
        ON stocks2018To2022.permno = stocksMarketEquity.permno
        AND stocks2018To2022.reference_date = stocksMarketEquity.reference_date
        '''
stocks2018To2022 = pd.read_sql(query, conn)
stocks2018To2022.drop('index', inplace = True, axis = 1)
stocks2018To2022 = stocks2018To2022.dropna()
stocks2018To2022 = stocks2018To2022.reset_index()
stocks2018To2022.drop('index', inplace = True, axis = 1)

In [23]:
#Calculate BM ratio
stocks2018To2022['bm_ratio'] = stocks2018To2022['book_value']/stocks2018To2022['market_equity']

#Add market cap of June of year y (of reference date) (used for value-weighted returns)
stocks2018To2022['date'] = pd.to_datetime(stocks2018To2022['date'])
stocksWeight = stocks2018To2022[stocks2018To2022['date'].dt.month == 6]
stocks2018To2022.to_sql("stocks2018To2022", conn, if_exists = 'replace')
stocksWeight.to_sql("stocksWeight", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT stocks2018To2022.*, stocksWeight.market_cap as market_cap_weight
        FROM stocks2018To2022
        LEFT JOIN stocksWeight
        ON stocks2018To2022.permno = stocksWeight.permno
        AND stocks2018To2022.reference_date = stocksWeight.reference_date
        '''
stocks2018To2022 = pd.read_sql(query, conn)
stocks2018To2022.drop('index', inplace = True, axis = 1)

In [24]:
#filter data
stocks2018To2022.replace([np.inf, -np.inf], np.nan, inplace = True)
stocks2018To2022 = stocks2018To2022.dropna()
stocks2018To2022['date'] = pd.to_datetime(stocks2018To2022['date'])
stocks2018To2022 = stocks2018To2022[stocks2018To2022['date'] >= '2018-01-01 00:00:00']

In [25]:
#Size Sort

#Get NYSE stocks' sizes in June of each year
stocks2018To2022['date'] = pd.to_datetime(stocks2018To2022['date'])
sizeBreakpoints = stocks2018To2022[(stocks2018To2022['date'].dt.month == 6) & (stocks2018To2022['exchange'] == "NYSE")]
sizeBreakpoints.to_sql("sizeBreakpoints", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT permno, reference_date, market_cap
        FROM sizeBreakpoints
        '''
sizeBreakpoints = pd.read_sql(query, conn)
sizeBreakpoints['reference_date'] = pd.to_datetime(sizeBreakpoints['reference_date'])
g = sizeBreakpoints.groupby(['reference_date'])

In [30]:
# Create new dataframe to sort sizes
sizeSorts = stocks2018To2022[(stocks2018To2022['date'].dt.month == 6)].copy()

# Initialize 'size_median' column
sizeSorts['size_median'] = None

# Calculate median size for each year
for i in sizeSorts.index:
    reference_date = sizeSorts.loc[i, 'reference_date']
    group_reference = g.get_group(reference_date)
    size_median = group_reference['market_cap'].quantile(0.5)
    sizeSorts.loc[i, 'size_median'] = size_median


In [33]:
sizeSorts['size_portfolio'] = ""
for i in sizeSorts.index:
    if sizeSorts.loc[i, 'market_cap'] > sizeSorts.loc[i, 'size_median']:
        sizeSorts.loc[i, 'size_portfolio'] = "B"
    else:
        sizeSorts.loc[i, 'size_portfolio'] = "S"

In [35]:
#Left join size portfolio back into stocks dataframe
stocks2018To2022.to_sql("stocks2018To2022", conn, if_exists = 'replace')
sizeSorts.to_sql("sizeSorts", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT stocks2018To2022.*, sizeSorts.size_portfolio
        FROM stocks2018To2022
        LEFT JOIN sizeSorts
        ON stocks2018To2022.reference_date = sizeSorts.reference_date
        AND stocks2018To2022.permno = sizeSorts.permno
        '''
stocks2018To2022 = pd.read_sql(query, conn)
stocks2018To2022.drop('index', inplace = True, axis = 1)

print(stocks2018To2022)
print(stocks2018To2022.columns)

        permno                 date exchange       ret    market_cap  \
0        10026  2019-06-28 00:00:00   NASDAQ     0.373  3.030689e+03   
1        10026  2019-07-31 00:00:00   NASDAQ   15.4644  3.501411e+03   
2        10026  2019-08-30 00:00:00   NASDAQ    3.8851  3.637443e+03   
3        10026  2019-09-30 00:00:00   NASDAQ   -0.2901  3.627840e+03   
4        10026  2019-10-31 00:00:00   NASDAQ   -0.6458  3.604410e+03   
...        ...                  ...      ...       ...           ...   
115641   93436  2021-11-30 00:00:00   NASDAQ    2.7612  1.149642e+06   
115642   93436  2021-12-31 00:00:00   NASDAQ   -7.6855  1.092218e+06   
115643   93436  2022-01-31 00:00:00   NASDAQ  -11.3609  9.681319e+05   
115644   93436  2022-02-28 00:00:00   NASDAQ   -7.0768  8.996190e+05   
115645   93436  2022-03-31 00:00:00   NASDAQ      -100  1.113736e+06   

             reference_date     gvkey             datadate  book_value  \
0       2019-06-01 00:00:00   12825.0  2018-09-30 00:00:00   

In [36]:
### Value sort ###
stocks2018To2022['date'] = pd.to_datetime(stocks2018To2022['date'])
valueBreakpoints = stocks2018To2022[(stocks2018To2022['date'].dt.month == 6) & (stocks2018To2022['exchange'] == "NYSE")]
valueBreakpoints.to_sql("valueBreakpoints", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT permno, reference_date, bm_ratio
        FROM valueBreakpoints
        '''
valueBreakpoints = pd.read_sql(query, conn)
valueBreakpoints['reference_date'] = pd.to_datetime(valueBreakpoints['reference_date'])
g = valueBreakpoints.groupby(['reference_date'])

In [37]:
valueSorts = stocks2018To2022[(stocks2018To2022['date'].dt.month == 6)].copy()

#Calculate 30th adn 70th quantile of book values
valueSorts['value_q30'] = ""
valueSorts['value_q70'] = ""
for i in valueSorts.index:
    valueSorts.loc[i, 'value_q30'] = g.get_group(g.get_group(valueSorts.loc[i, 'reference_date']).reset_index().loc[0, 'reference_date'])['bm_ratio'].quantile(0.3)
    valueSorts.loc[i, 'value_q70'] = g.get_group(g.get_group(valueSorts.loc[i, 'reference_date']).reset_index().loc[0, 'reference_date'])['bm_ratio'].quantile(0.7)

In [39]:
valueSorts['value_portfolio'] = ""
for i in valueSorts.index:
    if valueSorts.loc[i, 'bm_ratio'] > valueSorts.loc[i, 'value_q70']:
        valueSorts.loc[i, 'value_portfolio'] = "H"
    elif valueSorts.loc[i, 'bm_ratio'] > valueSorts.loc[i, 'value_q30']:
        valueSorts.loc[i, 'value_portfolio'] = "M"
    else:
        valueSorts.loc[i, 'value_portfolio'] = "L"

In [42]:
#left Join value portfolio back into stocks dataframe
stocks2018To2022.to_sql("stocks2018To2022", conn, if_exists = 'replace')
valueSorts.to_sql("valueSorts", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT stocks2018To2022.*, valueSorts.value_portfolio
        FROM stocks2018To2022
        LEFT JOIN valueSorts
        ON stocks2018To2022.reference_date = valueSorts.reference_date
        AND stocks2018To2022.permno = valueSorts.permno
        '''
stocks2018To2022 = pd.read_sql(query, conn)
stocks2018To2022.drop('index', inplace = True, axis = 1)

print(stocks2018To2022)
print(stocks2018To2022.columns)

        permno                 date exchange       ret    market_cap  \
0        10026  2019-06-28 00:00:00   NASDAQ     0.373  3.030689e+03   
1        10026  2019-07-31 00:00:00   NASDAQ   15.4644  3.501411e+03   
2        10026  2019-08-30 00:00:00   NASDAQ    3.8851  3.637443e+03   
3        10026  2019-09-30 00:00:00   NASDAQ   -0.2901  3.627840e+03   
4        10026  2019-10-31 00:00:00   NASDAQ   -0.6458  3.604410e+03   
...        ...                  ...      ...       ...           ...   
115641   93436  2021-11-30 00:00:00   NASDAQ    2.7612  1.149642e+06   
115642   93436  2021-12-31 00:00:00   NASDAQ   -7.6855  1.092218e+06   
115643   93436  2022-01-31 00:00:00   NASDAQ  -11.3609  9.681319e+05   
115644   93436  2022-02-28 00:00:00   NASDAQ   -7.0768  8.996190e+05   
115645   93436  2022-03-31 00:00:00   NASDAQ      -100  1.113736e+06   

             reference_date     gvkey             datadate  book_value  \
0       2019-06-01 00:00:00   12825.0  2018-09-30 00:00:00   

In [120]:
#Construct factors portfolio

#Calculate weighted returns
stocks2018To2022['date'] = pd.to_datetime(stocks2018To2022['date'])
valueWeightedRet = stocks2018To2022.groupby(['date', 'size_portfolio', 'value_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights = pd.to_numeric(x['market_cap_weight'])))

In [121]:
stocks2018To2022.to_sql("stocks2018To2022", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT date
        FROM stocks2018To2022
        '''
factors = pd.read_sql(query, conn)
factors['date'] = pd.to_datetime(factors['date'])

In [122]:
#Add smb portfolios
factors['smb'] = ""
for i in factors.index:
    dateOfData = factors.loc[i, 'date']
    factors.loc[i, 'smb'] = np.average(valueWeightedRet[dateOfData]["S"]) - np.average(valueWeightedRet[dateOfData]["B"])

In [124]:
print(valueWeightedRet)

date        size_portfolio  value_portfolio
2019-06-28  B               H                   8.649902
                            L                   7.869751
                            M                   6.067738
            S               H                   8.160416
                            L                   8.071897
                                                 ...    
2022-03-31  B               L                 -99.935768
                            M                 -99.957204
            S               H                 -99.531869
                            L                 -99.547837
                            M                 -99.864699
Length: 204, dtype: float64


In [125]:
#Change indexing of valueWeightedRet for HML
valueWeightedRet = stocks2018To2022.groupby(['date', 'value_portfolio', 'size_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights = pd.to_numeric(x['market_cap_weight'])))

# Add hml portfolios
factors['hml'] = ""
for i in factors.index:
    dateOfData = factors.loc[i, 'date']
    factors.loc[i, 'hml'] = np.average(valueWeightedRet[dateOfData]["H"]) - np.average(valueWeightedRet[dateOfData]["L"])

In [129]:
#Add mkt (monthly average return) across all stocks
factors['mkt'] = ""
weightedAvgMonthlyRet = stocks2018To2022.groupby('date').apply(lambda x: np.average(pd.to_numeric(x.ret), weights = pd.to_numeric(x.market_cap_weight)))
for i in factors.index:
    dateOfData = factors.loc[i, 'date']
    factors.loc[i, 'mkt'] = weightedAvgMonthlyRet[dateOfData]

In [130]:
#Filter out last month (ret = -100% due to missing dlret data)
factors['date'] = pd.to_datetime(factors['date'])
factors = factors[factors['date'] < "2022-3-31"]

#Parse relevant variables
factors.to_sql("factors", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT date, mkt, smb, hml
        FROM factors
        '''
factors = pd.read_sql(query, conn)

In [132]:
#sort by date
factors['date'] = pd.to_datetime(factors['date'])
factors = factors.sort_values(by = ['date'], ascending = True)

#Reindex
factors = factors.reset_index()
factors.drop('index', inplace = True, axis = 1)

print(factors)

         date        mkt       smb        hml
0  2019-06-28   7.442855  0.725381   0.434335
1  2019-07-31   1.451108 -2.028603   0.066145
2  2019-08-30  -2.374261 -2.802037  -4.746700
3  2019-09-30   1.867534  0.045370   5.694920
4  2019-10-31   2.251724 -0.015698  -2.206578
5  2019-11-29   4.040230  0.988982  -2.415970
6  2019-12-31   2.898891  1.488209   2.249337
7  2020-01-31  -0.200422 -3.625021  -6.096775
8  2020-02-28  -8.209507 -0.109146  -3.377045
9  2020-03-31 -14.038569 -7.288279 -12.349423
10 2020-04-30  14.391044  7.311321   2.131972
11 2020-05-29   5.059311  2.698112  -5.193029
12 2020-06-30   3.238252  4.809586  -2.446018
13 2020-07-31   5.810872 -1.980116  -1.637731
14 2020-08-31   7.379208  0.021343  -2.144586
15 2020-09-30  -3.638041 -0.115687  -2.046305
16 2020-10-30  -1.998925  3.976551   3.936080
17 2020-11-30  12.439364  7.063797   2.333686
18 2020-12-31   4.452327  5.474246  -2.392397
19 2021-01-29  -0.258749  6.875618   1.461469
20 2021-02-26   3.000820  4.254631

In [151]:
#import corresponding factors from Ken French's Study
file = filePath + "/FamaFrenchFiveFactorsMonthlyCopy.csv"
factorsFF = pd.read_csv(file)
print(factorsFF)
print(factors.columns)

       Date  Mkt_RF   SMB   HML   RMW   CMA    RF
0    196307   -0.39 -0.41 -0.97  0.68 -1.18  0.27
1    196308    5.07 -0.80  1.80  0.36 -0.35  0.25
2    196309   -1.57 -0.52  0.13 -0.71  0.29  0.27
3    196310    2.53 -1.39 -0.10  2.80 -2.01  0.29
4    196311   -0.85 -0.88  1.75 -0.51  2.24  0.27
..      ...     ...   ...   ...   ...   ...   ...
706  202205   -0.34 -0.06  8.41  1.44  3.98  0.03
707  202206   -8.43  1.30 -5.97  1.85 -4.70  0.06
708  202207    9.57  1.88 -4.10  0.68 -6.94  0.08
709  202208   -3.78  1.51  0.31 -4.80  1.31  0.19
710  202209   -9.36 -0.96  0.05 -1.40 -0.82  0.19

[711 rows x 7 columns]
Index(['date', 'mkt', 'smb', 'hml', 'mkt_ff', 'smb_ff', 'hml_ff'], dtype='object')


In [152]:
#clean FF dataframe

#change column names
factorsFF.to_sql("factorsFF", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT Date as date, Mkt_RF - RF as mkt_ff, SMB as smb_ff, HML as hml_ff
        FROM factorsFF
        '''

factorsFF = pd.read_sql(query, conn)
print(factorsFF)

       date  mkt_ff  smb_ff  hml_ff
0    196307   -0.66   -0.41   -0.97
1    196308    4.82   -0.80    1.80
2    196309   -1.84   -0.52    0.13
3    196310    2.24   -1.39   -0.10
4    196311   -1.12   -0.88    1.75
..      ...     ...     ...     ...
706  202205   -0.37   -0.06    8.41
707  202206   -8.49    1.30   -5.97
708  202207    9.49    1.88   -4.10
709  202208   -3.97    1.51    0.31
710  202209   -9.55   -0.96    0.05

[711 rows x 4 columns]


In [153]:
#Convert variables to correct type
factorsFF['mkt_ff'] = pd.to_numeric(factorsFF['mkt_ff'], errors = 'coerce')
factorsFF['smb_ff'] = pd.to_numeric(factorsFF['smb_ff'], errors = 'coerce')
factorsFF['hml_ff'] = pd.to_numeric(factorsFF['hml_ff'], errors = 'coerce')

In [154]:
#Add month column to factors and factorsFF to prepare for left join
factors['date'] = pd.to_datetime(factors['date'])
factors['month'] = factors['date'].dt.month
factorsFF['month'] = ""
for i in factorsFF.index:
    factorsFF.loc[i, 'month'] = int(str(factorsFF.loc[i, 'date'])[-2] + str(factorsFF.loc[i, 'date'])[-1])

In [155]:
#Add year column to factors and factorsFF to prepare for left join
factors['year'] = factors['date'].dt.year
for i in factorsFF.index:
    factorsFF.loc[i, 'year'] = int(str(factorsFF.loc[i, 'date'])[0:4])


In [156]:
#left join FF results info factors
factors.to_sql("factors", conn, if_exists = 'replace')
factorsFF.to_sql("factorsFF", conn, if_exists = 'replace')
query = '''
        SELECT factors.date, factors.mkt, factors.smb, factors.hml, 
                factorsFF.mkt_ff, factorsFF.smb_ff, factorsFF.hml_ff
        FROM factors
        LEFT JOIN factorsFF
        ON factors.month = factorsFF.month
        AND factors.year = factorsFF.year
        '''
factors = pd.read_sql(query, conn)
print(factors)

                   date        mkt       smb        hml  mkt_ff  smb_ff  \
0   2019-06-28 00:00:00   7.442855  0.725381   0.434335    6.75    0.36   
1   2019-07-31 00:00:00   1.451108 -2.028603   0.066145    1.00   -1.78   
2   2019-08-30 00:00:00  -2.374261 -2.802037  -4.746700   -2.74   -3.23   
3   2019-09-30 00:00:00   1.867534  0.045370   5.694920    1.25    0.27   
4   2019-10-31 00:00:00   2.251724 -0.015698  -2.206578    1.90    0.26   
5   2019-11-29 00:00:00   4.040230  0.988982  -2.415970    3.75    0.44   
6   2019-12-31 00:00:00   2.898891  1.488209   2.249337    2.63    0.97   
7   2020-01-31 00:00:00  -0.200422 -3.625021  -6.096775   -0.24   -4.37   
8   2020-02-28 00:00:00  -8.209507 -0.109146  -3.377045   -8.25    0.03   
9   2020-03-31 00:00:00 -14.038569 -7.288279 -12.349423  -13.52   -8.31   
10  2020-04-30 00:00:00  14.391044  7.311321   2.131972   13.65    2.59   
11  2020-05-29 00:00:00   5.059311  2.698112  -5.193029    5.57    1.98   
12  2020-06-30 00:00:00  

In [157]:
#Compare our results to FF results
factors.describe(percentiles = [0.1, 0.25, 0.5, 0.75, 0.9])

,mkt,smb,hml,mkt_ff,smb_ff,hml_ff
count,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000
mean,1.762842,0.236766,0.177142,1.580606,0.045758,0.069394
std,5.501896,3.656205,4.815404,5.409492,3.370821,5.121531
min,-14.038569,-7.288279,-12.349423,-13.520000,-8.310000,-13.970000
10%,-4.175650,-3.555314,-5.103763,-4.224000,-3.886000,-4.864000
25%,-1.767043,-2.373196,-2.392397,-1.550000,-1.780000,-2.170000
50%,2.826594,0.021343,-0.061248,2.450000,0.030000,-0.720000
75%,4.452327,1.966713,3.679564,4.620000,1.970000,3.040000
90%,7.177614,5.341314,5.703291,6.730000,4.618000,7.012000
max,14.391044,7.311321,12.955060,13.650000,7.120000,12.750000


In [162]:
#kolmogorov-Smirnov test to see if data has statistically significant differences
#mkt
print(ks_2samp(factors['mkt'], factors['mkt_ff']))
#smb
print(ks_2samp(factors['smb'], factors['smb_ff']))
#hml
print(ks_2samp(factors['hml'], factors['hml_ff']))

#first number is max difference
#second value is confidence level of the statistic


KstestResult(statistic=0.12121212121212122, pvalue=0.9725172787255439, statistic_location=2.78, statistic_sign=-1)
KstestResult(statistic=0.09090909090909091, pvalue=0.9994866573702369, statistic_location=4.79, statistic_sign=-1)
KstestResult(statistic=0.15151515151515152, pvalue=0.851010941578351, statistic_location=-0.16, statistic_sign=-1)


In [163]:
#compute correlations between variables
factors.corr(method = 'pearson')

C:\Users\aaron\AppData\Local\Temp\ipykernel_2200\803550743.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  factors.corr(method = 'pearson')


,mkt,smb,hml,mkt_ff,smb_ff,hml_ff
mkt,1.000000,0.491615,0.148353,0.997636,0.393338,0.119082
smb,0.491615,1.000000,0.143544,0.500128,0.939055,0.151542
hml,0.148353,0.143544,1.000000,0.122890,0.345756,0.984622
mkt_ff,0.997636,0.500128,0.122890,1.000000,0.400966,0.094691
smb_ff,0.393338,0.939055,0.345756,0.400966,1.000000,0.377847
hml_ff,0.119082,0.151542,0.984622,0.094691,0.377847,1.000000
